In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
import re


def preproc(x):
    x = re.sub(r"<br\s*/?>", " ", x)  
    x = re.sub(r"[^a-zA-Z0-9\s]", "", x) 
    x = x.lower()  
    return x


df["review"] = df["review"].apply(preproc)
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative": 0})
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming te...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1
...,...,...
49995,i thought this movie did a down right good job...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,i am a catholic taught in parochial elementary...,0
49998,im going to have to disagree with the previous...,0


In [3]:
corpus = " ".join(df["review"].tolist())

In [4]:
from collections import Counter

vocab_size = 20000
word_freq = Counter(corpus.split())
most_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
most_freq_words = [word for word, freq in most_freq[:vocab_size]]
vocab = {word: i + 1 for i, word in enumerate(most_freq_words)}

In [5]:
import json
with open("vocab.json", "w") as f:
    json.dump(vocab, f)

In [6]:
def encode(text):
    return [vocab.get(word, 0) for word in text.split()]

df["review"] = df["review"].apply(encode)
df

,review,sentiment
0,"[27, 4, 1, 75, 1944, 44, 1062, 11, 100, 143, 3...",1
1,"[3, 384, 116, 356, 1, 1338, 2937, 6, 51, 17935...",1
2,"[9, 192, 10, 12, 3, 384, 94, 5, 1120, 57, 19, ...",1
3,"[667, 214, 3, 233, 113, 3, 116, 435, 3556, 120...",0
4,"[0, 0, 109, 7, 1, 57, 4, 285, 6, 3, 2077, 1375...",1
...,...,...
49995,"[9, 192, 10, 16, 112, 3, 181, 200, 48, 286, 8,...",1
49996,"[77, 111, 77, 402, 77, 110, 2997, 963, 1, 626,...",0
49997,"[9, 227, 3, 3510, 4325, 7, 0, 8342, 4602, 31, ...",0
49998,"[140, 161, 5, 24, 5, 2964, 14, 1, 899, 912, 2,...",0


In [7]:
max_len = 200  
new_data = []

for i, row in df.iterrows():
    full_review = row["review"]
    sentiment = row["sentiment"]
    chunks = [full_review[j : j + max_len] for j in range(0, len(full_review), max_len)]

    for chunk in chunks:
        new_data.append([chunk, sentiment])

df = pd.DataFrame(new_data, columns=["review", "sentiment"])

In [8]:
max_len = 200
def pad(text):
    return text + [20001] * (max_len - len(text))

df["review"] = df["review"].apply(pad)
df["review"].apply(len)

0        200
1        200
2        200
3        200
4        200
        ... 
79534    200
79535    200
79536    200
79537    200
79538    200
Name: review, Length: 79539, dtype: int64

In [9]:
df.to_csv("data.csv", index=False)

In [10]:
import torch.nn as nn
import torch


class RNNModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to("cuda")
        x, h = self.rnn(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h.detach()

In [11]:
import torch.nn as nn
import torch


class LSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)

        if h is None:
            h = (
                torch.zeros(1, x.size(0), self.hidden_dim).to("cuda"),
                torch.zeros(1, x.size(0), self.hidden_dim).to("cuda"),
            )
        x, h = self.lstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h[0].detach(), h[1].detach())

In [12]:
import torch
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.hidden_dim = hidden_dim

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = torch.zeros(1, x.size(0), self.hidden_dim).to("cuda")
        x, h = self.gru(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, h.detach()

In [13]:
import torch
import torch.nn as nn

class BiLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h = (
                torch.zeros(2, x.size(0), self.bilstm.hidden_size).to("cuda"),
                torch.zeros(2, x.size(0), self.bilstm.hidden_size).to("cuda"),
            )
        x, h = self.bilstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h[0].detach(), h[1].detach())

In [14]:
import torch
import torch.nn as nn
import torch
import torch.nn as nn

class StackedLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, vocab_size):
        super(StackedLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim1, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        self.fc = nn.Linear(hidden_dim2, 1)  
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h1 = (
                torch.zeros(1, x.size(0), self.hidden_dim1, device=x.device),
                torch.zeros(1, x.size(0), self.hidden_dim1, device=x.device),
            )
            h2 = (
                torch.zeros(1, x.size(0), self.hidden_dim2, device=x.device),
                torch.zeros(1, x.size(0), self.hidden_dim2, device=x.device),
            )
        else:
            h1 = h[:2]
            h2 = h[2:]
        x, h1 = self.lstm1(x, h1)
        x, h2 = self.lstm2(x, h2)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h1[0].detach(), h1[1].detach(), h2[0].detach(), h2[1].detach())

In [15]:
import torch
import torch.nn as nn

class StackedBiLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, vocab_size):
        super(StackedBiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=20001)
        self.bilstm1 = nn.LSTM(embedding_dim, hidden_dim1, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(hidden_dim1 * 2, hidden_dim2, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim2 * 2, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)
        if h is None:
            h1 = (
                torch.zeros(2, x.size(0), self.bilstm1.hidden_size, device=x.device),
                torch.zeros(2, x.size(0), self.bilstm1.hidden_size, device=x.device),
            )
            h2 = (
                torch.zeros(2, x.size(0), self.bilstm2.hidden_size, device=x.device),
                torch.zeros(2, x.size(0), self.bilstm2.hidden_size, device=x.device),
            )
        else:
            h1 = h[:2] 
            h2 = h[2:]  
        x, h1 = self.bilstm1(x, h1)
        x, h2 = self.bilstm2(x, h2)
        x = x[:, -1, :]
        x = self.fc(x)
        return x, (h1[0].detach(), h1[1].detach(), h2[0].detach(), h2[1].detach())

In [16]:
import torch.optim as optim

batch_size = 256
seq_len = 200
learning_rate = 0.001
vocab_size = 20002
hidden_dim = 128

criterion = nn.BCEWithLogitsLoss()

In [17]:
from torch.utils.data import DataLoader, TensorDataset
train = TensorDataset(
    torch.tensor(df["review"][:50000].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][:50000].tolist(), dtype=torch.float32),
)
test = TensorDataset(
    torch.tensor(df["review"][50000:].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][50000:].tolist(), dtype=torch.float32),
)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [18]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch

def train_loop(model, bidir, epochs, model_name):
    model = model.to("cuda")

    writer = SummaryWriter(log_dir=f"runs/{model_name}")

    for epoch in range(epochs):
        hidden_state = None
        model.train()
        train_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        
        total_train_loss = 0
        correct_train = 0
        total_train = 0
    
        for i, (train_data, train_labels) in enumerate(train_tqdm):
            train_data, train_labels = train_data.to("cuda"), train_labels.to("cuda")
            if i == len(train_loader) - 1:  
                hidden_state = None
                
            optimizer.zero_grad()
            output, hidden_state = model(train_data, hidden_state)
            output = output.squeeze()
            if bidir:
                hidden_state = tuple(h.detach() for h in hidden_state)
            else:
                hidden_state = hidden_state.detach()
            loss = criterion(output, train_labels)
            loss.backward()
            optimizer.step()
    
            predictions = (torch.sigmoid(output) > 0.5).float()
            correct_train += (predictions == train_labels).sum().item()
            total_train += train_labels.size(0)
    
            total_train_loss += loss.item()
            train_tqdm.set_postfix(loss=loss.item())
    
        avg_train_loss = total_train_loss / len(train_loader)
        train_acc = correct_train / total_train

        writer.add_scalar(f"{model_name}/trainLoss", avg_train_loss, epoch+1)
        writer.add_scalar(f"{model_name}/trainAcc", train_acc, epoch+1)

        # ------------------- Validation Loop -------------------
        model.eval()
        val_tqdm = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]")
    
        total_val_loss = 0
        correct_val = 0
        total_val = 0
        hidden_state = None
        with torch.no_grad():
            for i, (val_data, val_labels) in enumerate(val_tqdm):
                val_data, val_labels = val_data.to("cuda"), val_labels.to("cuda")
                if i == len(test_loader) - 1:
                    hidden_state = None
                output, hidden_state = model(val_data)
                output = output.squeeze()
                if bidir:
                    hidden_state = tuple(h.detach() for h in hidden_state)
                else:
                    hidden_state = hidden_state.detach()
                loss = criterion(output, val_labels)
                total_val_loss += loss.item()
    
                predictions = (torch.sigmoid(output) > 0.5).float()
                correct_val += (predictions == val_labels).sum().item()
                total_val += val_labels.size(0)
    
                val_tqdm.set_postfix(loss=loss.item())
    
        avg_val_loss = total_val_loss / len(test_loader)
        val_acc = correct_val / total_val

        writer.add_scalar(f"{model_name}/valLoss", avg_val_loss, epoch+1)
        writer.add_scalar(f"{model_name}/valAcc", val_acc, epoch+1)
    
        print(
            f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}"
        )

    writer.close()  

In [19]:
model = RNNModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, False, 50, "RNN")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 225.90it/s, loss=0.721]


Epoch 1/50 | Train Loss: 0.6948 | Train Acc: 0.5014 | Val Loss: 0.6935 | Val Acc: 0.5085


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 219.55it/s, loss=0.703]


Epoch 2/50 | Train Loss: 0.6908 | Train Acc: 0.5177 | Val Loss: 0.6932 | Val Acc: 0.5102


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 225.89it/s, loss=0.681]


...

Epoch 48/50 | Train Loss: 0.5007 | Train Acc: 0.7731 | Val Loss: 0.7000 | Val Acc: 0.6716


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 225.91it/s, loss=0.698]


Epoch 49/50 | Train Loss: 0.4832 | Train Acc: 0.7835 | Val Loss: 0.6882 | Val Acc: 0.6889


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 208.82it/s, loss=0.672]

Epoch 50/50 | Train Loss: 0.4827 | Train Acc: 0.7856 | Val Loss: 0.6940 | Val Acc: 0.6829


In [20]:
model = LSTMModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "LSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 100.71it/s, loss=0.687]


Epoch 1/50 | Train Loss: 0.6917 | Train Acc: 0.5162 | Val Loss: 0.6914 | Val Acc: 0.5178


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 101.38it/s, loss=0.662]


Epoch 2/50 | Train Loss: 0.6834 | Train Acc: 0.5618 | Val Loss: 0.6731 | Val Acc: 0.5961


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 101.05it/s, loss=0.676]


...

Epoch 48/50 | Train Loss: 0.1187 | Train Acc: 0.9568 | Val Loss: 0.6256 | Val Acc: 0.8181


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 99.14it/s, loss=0.58]


Epoch 49/50 | Train Loss: 0.1180 | Train Acc: 0.9576 | Val Loss: 0.6228 | Val Acc: 0.8221


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 100.69it/s, loss=0.649]

Epoch 50/50 | Train Loss: 0.1121 | Train Acc: 0.9596 | Val Loss: 0.6690 | Val Acc: 0.8177


In [21]:
model = GRUModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, False, 50, "GRU")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 137.41it/s, loss=0.678]


Epoch 1/50 | Train Loss: 0.6915 | Train Acc: 0.5147 | Val Loss: 0.6853 | Val Acc: 0.5372


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 136.17it/s, loss=0.709]


Epoch 2/50 | Train Loss: 0.6741 | Train Acc: 0.5782 | Val Loss: 0.6477 | Val Acc: 0.6589


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 136.45it/s, loss=0.69]


...

Epoch 48/50 | Train Loss: 0.0233 | Train Acc: 0.9917 | Val Loss: 1.1703 | Val Acc: 0.8209


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:00<00:00, 133.48it/s, loss=0.912]


Epoch 49/50 | Train Loss: 0.0256 | Train Acc: 0.9908 | Val Loss: 1.1144 | Val Acc: 0.8207


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 82.62it/s, loss=0.96] 


Epoch 50/50 | Train Loss: 0.0209 | Train Acc: 0.9922 | Val Loss: 1.2133 | Val Acc: 0.8198


In [22]:
model = BiLSTMModel(64, 128, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "BiLSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 58.97it/s, loss=0.689]


Epoch 1/50 | Train Loss: 0.6927 | Train Acc: 0.5064 | Val Loss: 0.6914 | Val Acc: 0.5153


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 58.76it/s, loss=0.726]


Epoch 2/50 | Train Loss: 0.6860 | Train Acc: 0.5529 | Val Loss: 0.6926 | Val Acc: 0.5148


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 58.34it/s, loss=0.7]


...

Epoch 48/50 | Train Loss: 0.1676 | Train Acc: 0.9364 | Val Loss: 0.5934 | Val Acc: 0.8028


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 59.21it/s, loss=0.502]


Epoch 49/50 | Train Loss: 0.1599 | Train Acc: 0.9395 | Val Loss: 0.5861 | Val Acc: 0.8068


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 58.92it/s, loss=0.515]

Epoch 50/50 | Train Loss: 0.1521 | Train Acc: 0.9434 | Val Loss: 0.6397 | Val Acc: 0.8087


In [23]:
model = StackedLSTMModel(64, 128, 64, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "StackedLSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 86.54it/s, loss=0.696]


Epoch 1/50 | Train Loss: 0.6918 | Train Acc: 0.5142 | Val Loss: 0.6886 | Val Acc: 0.5293


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 85.72it/s, loss=0.705]


Epoch 2/50 | Train Loss: 0.6937 | Train Acc: 0.5266 | Val Loss: 0.6955 | Val Acc: 0.4541


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 85.00it/s, loss=0.713]


...

Epoch 48/50 | Train Loss: 0.1807 | Train Acc: 0.9354 | Val Loss: 0.5049 | Val Acc: 0.8231


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 84.84it/s, loss=0.49]


Epoch 49/50 | Train Loss: 0.1798 | Train Acc: 0.9352 | Val Loss: 0.5017 | Val Acc: 0.8242


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:01<00:00, 84.88it/s, loss=0.635]

Epoch 50/50 | Train Loss: 0.1663 | Train Acc: 0.9417 | Val Loss: 0.5507 | Val Acc: 0.8186


In [24]:
model = StackedBiLSTMModel(64, 128, 64, 20002)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loop(model, True, 50, "StackedBiLSTM")

Epoch 1/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 44.47it/s, loss=0.694]


Epoch 1/50 | Train Loss: 0.6919 | Train Acc: 0.5129 | Val Loss: 0.6890 | Val Acc: 0.5269


Epoch 2/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 42.66it/s, loss=0.715]


Epoch 2/50 | Train Loss: 0.6913 | Train Acc: 0.5241 | Val Loss: 0.6935 | Val Acc: 0.5072


Epoch 3/50 [Val]: 100%|██████████| 116/116 [00:03<00:00, 36.64it/s, loss=0.706]


...

Epoch 48/50 | Train Loss: 0.0842 | Train Acc: 0.9701 | Val Loss: 0.7435 | Val Acc: 0.8174


Epoch 49/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 43.75it/s, loss=0.719]


Epoch 49/50 | Train Loss: 0.0860 | Train Acc: 0.9693 | Val Loss: 0.7083 | Val Acc: 0.8270


Epoch 50/50 [Val]: 100%|██████████| 116/116 [00:02<00:00, 43.98it/s, loss=0.653]

Epoch 50/50 | Train Loss: 0.0813 | Train Acc: 0.9718 | Val Loss: 0.6949 | Val Acc: 0.8301
